# A simple implementation of skipgrams with negative sampling
Author: Pierre Nugues

Adapted from _Skipgrams with negative sampling from Distributed Representations of Words and Phrases and their Compositionality_ by Mikolov et al. 2013.

The imports

In [1]:
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, Lambda, Average, GlobalAveragePooling1D, Dot, Input, Reshape, Activation
import regex as re
import os
from tensorflow.keras.utils import to_categorical
import numpy as np
from scipy.spatial.distance import cosine
from tqdm import tqdm
from random import shuffle, randint
from collections import Counter
import math, random

## Parameters

The embedding size, context size, and negative counts

In [2]:
embedding_dim = 100
w_size = 2
c_size = w_size * 2 + 1
K_NEG = 5
t = 1e-3
power = 0.75

## The Corpus

We select a dataset and execute locally or on colab

In [3]:
dataset = 'homer'  # 'homer' dickens' 'selma' 'big'
colab = False # On my machine or on colab

In [4]:
if colab:
    BASE_PATH = '/content/drive/My Drive/Colab Notebooks/'
else:
    BASE_PATH = '../../../'

In [5]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive')

We read the files from a folder

In [6]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files


def load_corpus(path):
    files = get_files(path, 'txt')
    files = [path + file for file in files]
    print(files)
    text = ''
    for file in files:
        text += open(file).read()
    return text

In [7]:
if dataset == 'homer':
    #text = 'Sing, O goddess, the anger of Achilles son of Peleus'.lower()
    text1 = open(BASE_PATH + 'corpus/iliad.mb.txt', encoding='utf-8').read().lower()
    text2 = open(BASE_PATH + 'corpus/odyssey.mb.txt', encoding='utf-8').read().lower()
    text = text1 + text2
    test_words = ['he', 'she', 'ulysses', 'penelope', 'achaeans', 'trojans']
if dataset == 'dickens':
    path = BASE_PATH + 'corpus/Dickens/'
    text = load_corpus(path)
    test_words = ['he', 'she', 'paris', 'london', 'table', 'rare', 'monday', 'sunday', 'man', 'woman', 'king', 'queen', 'boy',
                  'girl']
elif dataset == 'selma':
    path = BASE_PATH + 'corpus/Selma/'
    text = load_corpus(path)
    test_words = ['han', 'hon', 'att', 'bord', 'bordet', 'måndag', 'söndag', 'man', 'kvinna', 'kung', 'drottning',
                  'pojke', 'flicka']
elif dataset == 'big':
    path = BASE_PATH + 'corpus/Dickens/'
    text = load_corpus(path)
    path = BASE_PATH + 'corpus/Norvig/'
    text += load_corpus(path)
    test_words = ['he', 'she', 'paris', 'london', 'table', 'rare', 'monday', 'sunday', 'man', 'woman', 'king', 'queen', 'boy',
                  'girl']

## Processing the Corpus

### Tokenizing

We set all the text in lowercase

In [8]:
text = text.lower()
word_seq = re.findall('\p{L}+', text)
word_seq[:5]

['book', 'i', 'the', 'quarrel', 'between']

### Downsampling

We can downsample the frequent words. We first count the words. We will have to count them again after sampling

In [9]:
counts = Counter(word_seq)

In [ ]:
counts['the']

In [ ]:
counts['penelope']

In [12]:
discard_probs = dict(counts)
word_cnt = sum(discard_probs.values())

In [14]:
for key in discard_probs:
    discard_probs[key] = max(0, 1 - math.sqrt(t/(discard_probs[key]/word_cnt)))

In [15]:
discard_probs['the']

0.8690560952429589

In [20]:
discard_probs['he']

0.7602888379452187

In [25]:
subsampled_word_seq = []
for word in word_seq:
    if discard_probs[word] < np.random.random():
        subsampled_word_seq += [word]

In [26]:
word_seq = subsampled_word_seq

### Counting the words

In [27]:
counts = Counter(word_seq)

In [30]:
word_cnt = sum(counts.values())
word_cnt

172113

We extract the unique words

In [31]:
unique_words = sorted(list(counts.keys()))
unique_words[:10]

['a',
 'abantes',
 'abarbarea',
 'abas',
 'abate',
 'abated',
 'abetting',
 'abhorred',
 'abians',
 'abide']

In [32]:
vocab_size = len(unique_words)
vocab_size

9725

### Indices

And we create indices

In [33]:
word2idx = {word: i for (i, word) in enumerate(unique_words)}
idx2word = {v: k for k, v in word2idx.items()}
#word2idx

We map the words to their indices and we get the sequence of word indices

In [34]:
widx_seq = list(map(word2idx.get, word_seq))
widx_seq[:5]

[1037, 6666, 897, 187, 67]

### Power transform

We apply a power tranform to a list of counts and we return power transformed probabilities

In [35]:
def power_transform(counts, power):
    trfmd_probs = dict()
    for word in counts:
        trfmd_probs[word] = math.pow(counts[word], power)
    sum_probs = sum(trfmd_probs.values())
    for word in trfmd_probs:
        trfmd_probs[word] /= sum_probs
    return trfmd_probs

In [36]:
trfmd_probs = power_transform(counts, power)

We build the index and proability lists for the random choice function

In [37]:
trfmd_probs_idx = {word2idx[k]: v for k, v in trfmd_probs.items()}

In [38]:
draw_idx, probs = zip(*trfmd_probs_idx.items())

Drawing words

In [39]:
random.choices(draw_idx, weights=probs, k=K_NEG * 2 * w_size)

[4115,
 8488,
 2652,
 2470,
 7191,
 8772,
 6421,
 6755,
 3305,
 9419,
 5719,
 6634,
 2627,
 5047,
 9340,
 2999,
 8957,
 9104,
 3883,
 3970]

## The pairs

For all the words, we form positive and negative pairs. We extract the context words of a word from its neighbors in the word sequence to form the positive pairs and at random to form the negative ones.

In [40]:
positive_pairs = []
negative_pairs = []
for idx, widx in tqdm(enumerate(widx_seq)):
    # We create the start and end indices as in range(start, end)
    start_idx = max(0, idx - w_size)
    end_idx = min(idx + w_size + 1, len(widx_seq))
    # We create pairs from the left context: start_idx -> idx
    for c_idx in range(start_idx, idx):
        positive_pairs += [(widx_seq[idx], widx_seq[c_idx])]
        negative_pairs += [(widx_seq[idx], neg) for neg in random.choices(draw_idx, weights=probs, k=K_NEG)]
    # We create pairs from the right context idx + 1 -> end_idx
    for c_idx in range(idx + 1, end_idx):
        positive_pairs += [(widx_seq[idx], widx_seq[c_idx])]
        negative_pairs += [(widx_seq[idx], neg) for neg in random.choices(draw_idx, weights=probs, k=K_NEG)]

172113it [04:02, 709.64it/s] 


In [ ]:
pairs = positive_pairs + negative_pairs

We build two inputs: The left input is the input word and the right one is a context word.

In [ ]:
X = np.array(pairs)
X_i = X[:, 0]
X_c = X[:, 1]

In [ ]:
y = [1] * len(positive_pairs) + [0] * len(negative_pairs)
y = np.array(y)

## The Architecture

And now the architecture

In [ ]:
i_word = Input(shape=(1,))
i_embedding = Embedding(vocab_size, embedding_dim, input_length=1)(i_word)

c_word = Input(shape=(1,))
c_embedding = Embedding(vocab_size, embedding_dim, input_length=1)(c_word)

dot_prod = Dot(axes=-1)([i_embedding, c_embedding])
output = Dense(1, activation='sigmoid')(dot_prod)
model = Model([i_word, c_word], output)
model.summary()

In [ ]:
def loss_neg(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    log_y_pred_1 = tf.math.log(y_pred)
    log_y_pred_0 = tf.math.log(1.0 - y_pred)
    loss = tf.math.add(tf.math.multiply(y_true, log_y_pred_1),
                       tf.math.multiply(1.0 - y_true, log_y_pred_0))
    loss = -tf.reduce_sum(loss)
    return loss

In [ ]:
model.compile(loss=loss_neg, optimizer='rmsprop')

In [ ]:
model.fit([X_i, X_c], y, epochs=2)

In [ ]:
def most_sim_vecs(vector, U, nbr_words=10):
    # Here cosine distance and not cosine
    # distance between equal vectors: 0. max distance: 2
    dist = [cosine(vector, U[i, :]) if np.any(U[i, :]) else 2
            for i in range(U.shape[0])]
    sorted_vectors = sorted(range(len(dist)), key=lambda k: dist[k])
    return sorted_vectors[1:nbr_words + 1]

In [ ]:
vectors = model.get_weights()[0]
most_sim_words = {}
for w in test_words:
    most_sim_words[w] = most_sim_vecs(vectors[word2idx[w]], vectors)
    most_sim_words[w] = list(map(idx2word.get, most_sim_words[w]))
    print(w, most_sim_words[w])

In [ ]:
"""
base 2 epochs
he ['i', 'you', 'a', 'his', 'him', 'as', 'with', 'that', 'for', 'in']
she ['her', 'who', 'no', 'or', 'their', 'this', 'be', 'your', 'one', 'said']
ulysses ['s', 'out', 'us', 'minerva', 'there', 'achilles', 'now', 'went', 'has', 'came']
penelope ['each', 'hands', 'gone', 'battle', 'looking', 'drink', 'telemachus', 'priam', 'same', 'fight']
achaeans ['ships', 'trojans', 'city', 'gods', 'sea', 'suitors', 'town', 'darkness', 'meanwhile', 'house']
trojans ['sea', 'achaeans', 'ships', 'suitors', 'city', 'getting', 'goddess', 'gods', 'house', 'before']
"""

In [ ]:
"""
power transform 2 epochs
he ['i', 'you', 'as', 'him', 'not', 'that', 'it', 'they', 'was', 'had']
she ['your', 'us', 'take', 'back', 'some', 's', 'may', 'go', 'let', 'which']
ulysses ['this', 'what', 'could', 'spoke', 'can', 'make', 'answered', 'made', 'hector', 'telemachus']
penelope ['find', 'let', 'than', 'think', 'noble', 'bring', 'back', 'himself', 'olympus', 'great']
achaeans ['among', 'gods', 'out', 'same', 'ships', 'city', 'where', 'house', 'gates', 'suitors']
trojans ['achaeans', 'ranks', 'rest', 'among', 'ground', 'others', 'suitors', 'island', 'dust', 'gates']
"""

In [ ]:
"""
power transform 2 epochs, neg loss
he ['you', 'i', 'it', 'cover', 'will', 'have', 'that', 'him', 'was', 'heal']
she ['ulysses', 'troy', 'did', 'long', 'jove', 'battle', 'very', 'put', 'her', 'think']
ulysses ['jove', 'be', 'battle', 'go', 'did', 'thus', 'after', 'think', 'friends', 'never']
penelope ['round', 'house', 'keep', 'host', 'strength', 'achaeans', 'presently', 'whom', 'servants', 'both']
achaeans ['into', 'house', 'other', 'gods', 'fire', 'both', 'city', 'host', 'king', 'back']
trojans ['into', 'other', 'about', 'themselves', 'away', 'sea', 'through', 'among', 'achaeans', 'back']"""

In [ ]:
"""
power transform 8 epochs, neg loss
he ['you', 'i', 'it', 'cover', 'will', 'have', 'that', 'him', 'was', 'heal']
she ['ulysses', 'troy', 'did', 'long', 'jove', 'battle', 'very', 'put', 'her', 'think']
ulysses ['jove', 'be', 'battle', 'go', 'did', 'thus', 'after', 'think', 'friends', 'never']
penelope ['round', 'house', 'keep', 'host', 'strength', 'achaeans', 'presently', 'whom', 'servants', 'both']
achaeans ['into', 'house', 'other', 'gods', 'fire', 'both', 'city', 'host', 'king', 'back']
trojans ['into', 'other', 'about', 'themselves', 'away', 'sea', 'through', 'among', 'achaeans', 'back']
"""